# Notebook experimenting with models available online

I have created a local file, **api_keys.py**, in the same directory as the notebook which contains variables with the api keys associated with the different services. You will need to do the same for this code to work. This will require creating accounts with [AssemblyAI](https://www.assemblyai.com) and [OpenAI](https://openai.com) to get your individual api keys.

Example for the **api_keys.py** file:

aai_api_key = {YOUR_ASSEMBLYAI_API_KEY}

You will also need install these packages:
- assemblyai
- langchain_community
- langchain_openai

In [1]:
import assemblyai as aai
from langchain_community.document_loaders import AssemblyAIAudioTranscriptLoader
from langchain_openai import OpenAI
from api_keys import aai_api_key, openai_api_key

In [2]:
# test journal stored under the filename "test_journal.m4a"
audio_file = "test_journal.m4a"

# configuration for the model performing the transcription
config = aai.TranscriptionConfig(
    sentiment_analysis = True,
    summarization=True
)

In [3]:
loader = AssemblyAIAudioTranscriptLoader(file_path=audio_file,
                                         api_key=aai_api_key,
                                         config=config)
docs = loader.load()

In [7]:
# docs[0].page_content
# docs[0].metadata['audio_duration']
# docs[0].metadata['confidence']
# docs[0].metadata['words'][0]
# docs[0].metadata['summary']
# docs[0].metadata['sentiment_analysis_results'][7]

In [9]:
pos, neu, neg = 0, 0, 0

# docs[0].metadata['sentiment_analysis']
for i in docs[0].metadata['sentiment_analysis_results']:
    sentiment = i['sentiment'].value
    if sentiment == "POSITIVE":
        pos += 1
    elif sentiment == "NEUTRAL":
        neu += 1
    elif sentiment == "NEGATIVE":
        neg += 1

print(pos, neu, neg)

7 3 3


Trying through AssemblyAI directly, rather than AssemblyAI through LangChain

In [10]:
aai.settings.api_key = aai_api_key

transcriber = aai.Transcriber()
transcript = transcriber.transcribe(audio_file, config)

In [13]:
# transcript.text
# transcript.words
# transcript.sentiment_analysis
# transcript.summary

In [ ]:
# use lemur llm from AssemblyAI to summarise the transcript into emotions - have to pay to use api
result = transcript.lemur.task(
    prompt="Summarise the feelings in this transcript into a few key words",
    temperature=0.5
)

In [ ]:
# possible to remove all data relating to request - good for controlling data
aai.Lemur.purge_request_data(result.request_id)

OpenAI below - need to pay to use api

In [14]:
llm = OpenAI(api_key=openai_api_key)

In [ ]:
prompt="Summarise the feelings in this transcript into a few key words"

llm.invoke(
    input = f"{prompt} : {docs[0].page_content}"
)

Proof of concepts completed so far:
- Speech-to-text conversion (transcription)
- Sentiment analsis
- Summary (kind of - not sure how useful the summary from AssemblyAI is)

Next:
- Integrate transcription with LLM to generate keywords for a summary / determine topics discussed / extract some emotion or feeling
- Analysis of audio (pitch, tone, loudness etc.)